In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 개선한 LSTM 2

In [4]:
# 데이터 로드 및 전처리
df = pd.read_csv('C:/Users/LG/Desktop/Junior/1학기/통계데이터 분석 공모전/merged_predict2.csv')
df['일시'] = pd.to_datetime(df['일시'], format='%m').dt.strftime('%m')
df = df.sort_values(by='일시')
df = df.fillna(0)
df.set_index('일시', inplace=True)

X = df[['검색건수', '관외 이동량', '숙박업소 개수', 'sns']]
y = df['방문자수']

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

In [6]:
# 시계열 데이터를 LSTM 입력 형식에 맞게 변환
def create_sequences(X, y, time_steps=30):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 30
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)

# 데이터셋 분할
split = int(len(X_seq) * 0.8)
X_train_seq, X_test_seq = X_seq[:split], X_seq[split:]
y_train_seq, y_test_seq = y_seq[:split], y_seq[split:]

X_train_rf, X_test_rf = X_scaled[:split], X_scaled[split:]
y_train_rf, y_test_rf = y_scaled[:split], y_scaled[split:]

# 조기 종료 콜백
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [7]:
# 3차원 데이터를 2차원으로 변환
X_train_seq_flat = X_train_seq.reshape(-1, X_train_seq.shape[-1])
X_test_seq_flat = X_test_seq.reshape(-1, X_test_seq.shape[-1])

# 1차원 배열로 변환
y_train_seq_flat = y_train_seq.flatten()
y_test_seq_flat = y_test_seq.flatten()

# DataFrame으로 변환
X_train_seq_df = pd.DataFrame(X_train_seq_flat)
X_test_seq_df = pd.DataFrame(X_test_seq_flat)
y_train_seq_series = pd.Series(y_train_seq_flat)
y_test_seq_series = pd.Series(y_test_seq_flat)

# CSV 파일로 저장
X_train_seq_df.to_csv('X_train_seq.csv', index=False)
X_test_seq_df.to_csv('X_test_seq.csv', index=False)
y_train_seq_series.to_csv('y_train_seq.csv', index=False, header=True)
y_test_seq_series.to_csv('y_test_seq.csv', index=False, header=True)

# 저장된 CSV 파일을 엑셀 파일로 합치기
csv_files = {
    'X_train_seq': 'X_train_seq.csv',
    'X_test_seq': 'X_test_seq.csv',
    'y_train_seq': 'y_train_seq.csv',
    'y_test_seq': 'y_test_seq.csv',
}

excel_file = 'Datasets1.xlsx'

try:
    with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
        for sheet_name, csv_file in csv_files.items():
            dff = pd.read_csv(csv_file)
            dff.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f'엑셀 파일 "{excel_file}"이 생성되었습니다.')

except FileNotFoundError:
    print(f'파일을 찾을 수 없습니다: {csv_file}')

except Exception as e:
    print(f'오류 발생: {e}')

엑셀 파일 "Datasets1.xlsx"이 생성되었습니다.


In [8]:
# 시계열 데이터를 LSTM 입력 형식에 맞게 변환
def create_sequences(X, y, time_steps=30):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 30

# 전체 데이터에 대해 시퀀스 생성
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)

In [9]:
# 데이터 분할 (훈련, 검증, 테스트)
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.15)
X_train_seq, y_train_seq = X_seq[:train_size], y_seq[:train_size]
X_val_seq, y_val_seq = X_seq[train_size:train_size+val_size], y_seq[train_size:train_size+val_size]
X_test_seq, y_test_seq = X_seq[train_size+val_size:], y_seq[train_size+val_size:]

# 비시퀀스 모델을 위한 데이터 분할
X_train, y_train = X_scaled[time_steps:train_size+time_steps], y_scaled[time_steps:train_size+time_steps]
X_val, y_val = X_scaled[train_size+time_steps:train_size+val_size+time_steps], y_scaled[train_size+time_steps:train_size+val_size+time_steps]
X_test, y_test = X_scaled[train_size+val_size+time_steps:], y_scaled[train_size+val_size+time_steps:]

# 시계열 교차검증 설정
tscv = TimeSeriesSplit(n_splits=5)

In [28]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import numpy as np

# # create_and_fit_model 함수 정의
# def create_and_fit_model(X_train, y_train, X_val, y_val, units, optimizer):
#     model = Sequential([
#         LSTM(units, input_shape=(X_train.shape[1], X_train.shape[2])),
#         Dense(1)
#     ])
#     model.compile(optimizer=optimizer, loss='mean_squared_error')
#     history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=50)
#     return model, history

# # 그리드 서치에 사용될 파라미터
# lstm_params = {'units': [50, 100], 'optimizer': ['adam', 'rmsprop']}

# # 모델 딕셔너리 설정
# models = {
#     'LSTM': (None, lstm_params)  # 모델 생성은 함수 내에서 처리
# }

# results = {}
# best_score = float('inf')
# best_params = None
# best_model = None

# # 그리드 서치 수행
# for units in lstm_params['units']:
#     for optimizer in lstm_params['optimizer']:
#         model, history = create_and_fit_model(X_train_seq, y_train_seq, X_val_seq, y_val_seq, units=units, optimizer=optimizer)
#         score = model.evaluate(X_val_seq, y_val_seq)
#         if score < best_score:
#             best_score = score
#             best_params = {'units': units, 'optimizer': optimizer}
#             best_model = model

# # 결과 저장
# results['LSTM'] = {
#     'Best Model': best_model, 
#     'Best Parameters': best_params, 
#     'History': history
# }

# # 예측 및 역스케일링
# y_pred = best_model.predict(X_test_seq)
# y_true = y_test_seq
# y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))
# y_true = scaler_y.inverse_transform(y_true.reshape(-1, 1))

# # 성능 지표 계산
# mse = mean_squared_error(y_true, y_pred)
# mae = mean_absolute_error(y_true, y_pred)
# nmae = mae / np.mean(np.abs(y_true))
# r2 = r2_score(y_true, y_pred)
# accuracy = r2 * 100

# # 성능 지표 결과 업데이트
# results['LSTM'].update({
#     'MSE': mse,
#     'MAE': mae,
#     'NMAE': nmae,
#     'R2': r2,
#     'Accuracy': accuracy
# })

# print(results['LSTM'])


C:\Users\LG\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0391 - val_loss: 0.0256
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0244 - val_loss: 0.0256
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0256 - val_loss: 0.0246
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0237 - val_loss: 0.0250
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0233 - val_loss: 0.0242
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0234 - val_loss: 0.0239
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0242 - val_loss: 0.0240
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0228 - val_loss: 0.0242
Epoch 9/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0248 - val_loss: 0.0243
Epoch 10/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0229 - val_loss: 0.0237
Epoch 11/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0259 - val_loss: 0.0241
Epoch 12/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0

In [29]:
# LSTM 모델 생성 및 훈련 함수
def create_and_fit_model(X_train, y_train, X_val, y_val, units=50, optimizer='adam', epochs=100, batch_size=32):
    model = Sequential([
        LSTM(units, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(1)
    ])

    model.compile(optimizer=optimizer, loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)
    return model, history

In [30]:
# 그리드 서치를 위한 파라미터 설정
lstm_params = {
    'units': [30, 50, 70],
    'optimizer': ['adam', 'rmsprop']
}


In [31]:
# 성능 평가 함수
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    nmae = calculate_nmae(y_true, y_pred, S)
    r2 = r2_score(y_true, y_pred)
    return mse, mae, nmae, r2

In [32]:
models = {
    'LSTM': (create_and_fit_model, lstm_params)
}

# 결과 저장을 위한 딕셔너리
results = {}

In [35]:
for name, (model_func, params) in models.items():
    print(f"Performing grid search for {name}...")

    if name in ['LSTM']:
        best_score = float('inf')
        best_params = None
        best_model = None
        best_history = None

        for units in params['units']:
            for optimizer in params['optimizer']:
                model, history = model_func(X_train_seq, y_train_seq, X_val_seq, y_val_seq,
                                            units=units, optimizer=optimizer)
                score = model.evaluate(X_val_seq, y_val_seq, verbose=0)
                print(f"Testing {name} with units={units}, optimizer={optimizer}, score={score}")

                if score < best_score:
                    best_score = score
                    best_params = {'units': units, 'optimizer': optimizer}
                    best_model = model
                    best_history = history

        results[name] = {'Best Model': best_model, 'Best Parameters': best_params, 'History': best_history}
    else:
        # Adjust as needed for other model types if added later
        pass

    # Evaluate the best model
    y_pred = best_model.predict(X_test_seq)
    y_true = y_test_seq

    # Inverse transform to original scale
    y_pred = scaler_y.inverse_transform(y_pred)
    y_true = scaler_y.inverse_transform(y_true)

    # Calculate performance metrics
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    accuracy = r2 * 100

    results[name].update({
        'MSE': mse,
        'MAE': mae,
        'R2': r2,
        'Accuracy': accuracy
    })

    print(f"Results for {name}: MSE={mse}, MAE={mae}, R2={r2}, Accuracy={accuracy}%")


Performing grid search for LSTM...


C:\Users\LG\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Testing LSTM with units=30, optimizer=adam, score=0.023543961346149445
Testing LSTM with units=30, optimizer=rmsprop, score=0.023534566164016724
Testing LSTM with units=50, optimizer=adam, score=0.02354712225496769
Testing LSTM with units=50, optimizer=rmsprop, score=0.023670662194490433
Testing LSTM with units=70, optimizer=adam, score=0.023693736642599106
Testing LSTM with units=70, optimizer=rmsprop, score=0.02351974882185459
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Results for LSTM: MSE=9486957705864.871, MAE=2380572.9082167833, R2=0.167546191628676, Accuracy=16.754619162867602%
